In [1]:
#from laspy.file import File
# #import laspy as lp
# #gpd.options.use_pygeos = True
# import pyvista as pv
# import pptk
import matplotlib.pyplot as plt
from shapely.geometry import Point
import pygeos
import rtree
import open3d as o3d
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from scipy.spatial import Delaunay
import math
from functools import reduce

In [2]:
# TODO: clean up

In [3]:
# read point cloud
data_folder = '/Users/philipp/Projects/PycharmProjects/point-cloud-volume/input/'
dataset = "points-32638.xyz"
npcloud = np.loadtxt(data_folder + dataset, delimiter=',')

crs = None
geometry = [Point(xyz) for xyz in npcloud]
geodf = gpd.GeoDataFrame(npcloud, crs=crs, geometry=geometry)
geodf.crs = {'init': 'epsg:32638'}
# geodf = geodf.to_crs("EPSG:4326")

/Users/philipp/Projects/PycharmProjects/point-cloud-volume/venv/lib/python3.9/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [25]:
# polygon vector
annotations = gpd.read_file('/Users/philipp/BPLA Dropbox/03 Planning/DN-Dumping Detection-1087/05_Output/Digitisation/Version-3/DD-lowest/boundary-reviewed/piles-combined-lowest-final-clipped-v2.1.geojson')
annotations = annotations.to_crs("EPSG:32638")
piles=piles.to_crs("EPSG:32638")
# piles['str_geom'] = piles['geometry'].apply(wkt.dumps)
# piles['FIDint'] = piles['fid'].astype('int64')
# len(piles)

NameError: name 'piles' is not defined

NameError: name 'file_list' is not defined

In [26]:
# Points in polygon
pointInPolys = gpd.tools.sjoin(geodf, annotations, predicate="within", how='left')
pointInPolysNA = pointInPolys.dropna(subset=['fid']).to_numpy()
# uniquePolys = pointInPolysNA.index_right.unique()

In [27]:
pointInPolysNA

array([[664710.9432641491, 2715806.585965889, 688.3612604738311, ...,
        33.747, 17.498, 17.498],
       [664711.6017343738, 2715806.593732364, 688.441260473831, ...,
        33.747, 17.498, 17.498],
       [664712.2602045987, 2715806.60149887, 688.561260473831, ...,
        33.747, 17.498, 17.498],
       ...,
       [664796.582637686, 2715806.9317188924, 689.081260473831, ...,
        35.31, 17.017, 17.017],
       [664797.2411081232, 2715806.9394894075, 688.9712604738311, ...,
        35.31, 17.017, 17.017],
       [664797.8995785619, 2715806.9472599537, 688.8712604738311, ...,
        35.31, 17.017, 17.017]], dtype=object)

In [ ]:
#subset for tests
nc1 = pointInPolys[pointInPolys.index_right == 1418]
nc1 = npcloud[pointInPolys.index_right == 1418, ]
nc1 = pointInPolys[pointInPolys['index_right'] == 1418, ]
nc1 = npcloud[pointInPolys.name == '32'][1]
#pointInPolys.id.unique()
npcloud[1]

In [ ]:
# run stats on the height
#stats_pt = pointInPolys.groupby('id')[2].agg(['max','min','mean'])
#stats_pt['range'] = stats_pt['max']-stats_pt['min']

In [ ]:

# merge
#result = pd.merge(annotations, stats_pt , left_on='id',right_index=True,how='outer')
nc1=temp
### ball pivot
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(nc1[:,:3])
#pcd.compute_convex_hull()
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(30)
o3d.visualization.draw_geometries([pcd])
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radii = [0.1*avg_dist, 0.5*avg_dist, 1*avg_dist, 2*avg_dist]
#radii = [0.005, 0.01, 0.02, 0.04]
r = o3d.utility.DoubleVector(radii)
BPmesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, r)
o3d.visualization.draw_geometries([pcd, BPmesh], mesh_show_back_face=True)
BPmesh.get_surface_area()
BPmesh.get_volume()

### voxel

### poisson mesh
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(base[:,:3])
pcd.colors = o3d.utility.Vector3dVector(base[:,3:6]/255)
#pcd.normals = o3d.utility.Vector3dVector(nc1[:,6:9])
pcd.estimate_normals()
#pcd.estimate_normals(search_param=o3d.pybind.geometry.KDTreeSearchParamHybrid(radius=0.1,max_nn= 30))
#mesh
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=40, width=0, scale=2, linear_fit=False)[0]
o3d.visualization.draw_geometries([pcd, poisson_mesh], mesh_show_back_face=True)
BPmesh.get_surface_area()
BPmesh.get_volume()


voxel_grid=o3d.geometry.VoxelGrid.create_from_point_cloud(pcd,voxel_size=0.05)


vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Bunny Visualize', width=800, height=600)

vis.add_geometry(poisson_mesh)
vis.run()
poisson_mesh.get_surface_area()
rec_mesh.get_volume()

distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radii = [0.1*avg_dist, 0.5*avg_dist, 1*avg_dist, 2*avg_dist]
#radii = [0.005, 0.01, 0.02, 0.04]
r = o3d.utility.DoubleVector(radii)
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, r)
#o3d.visualization.draw_geometries([pcd, rec_mesh], mesh_show_back_face=True)
poisson_mesh.get_surface_area()
poisson_mesh.get_volume()

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(nc1[:,:3])
pcd.colors = o3d.utility.Vector3dVector(nc1[:,3:6]/255)
#pcd.normals = o3d.utility.Vector3dVector(nc1[:,6:9])
pcd.estimate_normals(search_param=o3d.pybind.geometry.KDTreeSearchParamHybrid(radius=0.1,max_nn= 30))
#mesh
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=5.1, linear_fit=False)[0]







import pyvista as pv
"""`pc` is a `pyvista.PolyData` point cloud. The default arguments are abitrary"""
cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(nc1[:,:3])
cloud.estimate_normals()
trimesh,_ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(cloud, depth=8, width=0, scale=1.1, linear_fit=False)

v = np.asarray(trimesh.vertices)
f = np.array(trimesh.triangles)
f = np.c_[np.full(len(f), 3), f]
mesh = pv.PolyData(v, f)
meshclean= mesh.clean()
mesh.volume()
meshclean.plot(show_edges=True, line_width=3)

rec_mesh,_ = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, r)
vrec = np.asarray(rec_mesh.vertices)
frec = np.array(rec_mesh.triangles)
frec = np.c_[np.full(len(frec), 3), frec]
meshrec = pv.PolyData(vrec, frec)
meshclean= mesh.clean()
mesh.volume()
meshclean.plot(show_edges=True, line_width=3)

bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)
p_mesh_crop.get_surface_area()
p_mesh_crop.get_volume()


pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(nc1[:,:3])
pcd.colors = o3d.utility.Vector3dVector(nc1[:,3:6]/255)
#pcd.normals = o3d.utility.Vector3dVector(nc1[:,6:9])
pcd.estimate_normals(search_param=o3d.pybind.geometry.KDTreeSearchParamHybrid(radius=0.1,max_nn= 30))
#mesh
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=5.1, linear_fit=False)[0]
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)
p_mesh_crop.get_surface_area()
p_mesh_crop.get_volume()

o3d.io.write_triangle_mesh(data_folder+"bpa_mesh.ply", poisson_mesh)
o3d.io.write_triangle_mesh(data_folder+"p_mesh_c.ply", p_mesh_crop)

def lod_mesh_export(mesh, lods, extension, path):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh(path+"lod_"+str(i)+extension, mesh_lod)
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods

my_lods = lod_mesh_export(poisson_mesh, [100000,50000,10000,1000,100], ".ply", data_folder)
my_lods2 = lod_mesh_export(poisson_mesh, [8000,800,300], ".ply", data_folder)
o3d.visualization.draw_geometries([my_lods2[100]])

o3d.visualization.draw_geometries([pcd])
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

bpa_mesh = pcd.o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(o3d.utility.DoubleVector([radius, radius * 2]))

poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False)[0]


pcd.voxel_down_sample(pcd, voxel_size=0.02)
pcd.estimate_normals(pcd,s earch_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,max_nn=30))
downpcd = o3d.geometry.  voxel_down_sample(nc1, voxel_size=0.05)
o3d.visualization.draw_geometries([downpcd])
print("Recompute the normal of the downsampled point cloud")
pcd.estimate_normals(pcd,search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,max_nn=10))

a=pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    o3d.visualization.draw_geometries([downpcd])

nc1.view()
o3d.visualization.draw_geometries([pcd])

distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))

dec_mesh = mesh.simplify_quadric_decimation(100000)

dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

In [ ]:

import open3d as o3d

from shapely.geometry import Point
from open3d import voxel_down_sample,estimate_normals
#npcloud = np.loadtxt(data_folder + dataset, delimiter=',')


pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(nc1[:,:3])
pcd.compute_convex_hull()
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(30)

voxel_grid=o3d.geometry.VoxelGrid.create_from_point_cloud(pcd,voxel_size=0.01)

vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Bunny Visualize', width=800, height=600)

vis.add_geometry(voxel_grid)

vis.run()


distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radii = [0.1*avg_dist, 0.5*avg_dist, 1*avg_dist, 2*avg_dist]
#radii = [0.005, 0.01, 0.02, 0.04]
r = o3d.utility.DoubleVector(radii)
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, r)
#o3d.visualization.draw_geometries([pcd, rec_mesh], mesh_show_back_face=True)
rec_mesh.get_surface_area()
rec_mesh.get_volume()
